In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle
import numpy as np
import os

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Project Gutenberg's The Adventures  new.txt to Project Gutenberg's The Adventures  new.txt


In [3]:

# Load and preprocess your text data as before
file = open("Project Gutenberg's The Adventures  new.txt", "r", encoding="utf8")
lines = [line for line in file]
data = ' '.join(lines)
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
vocab_size = len(tokenizer.word_index) + 1

sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

X = np.array(X)
y = np.array(y)
y = to_categorical(y, num_classes=vocab_size)

embedding_dim = 50
lstm_units = 256

sequence_length = 3

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=sequence_length))
model.add(LSTM(lstm_units))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))

The Length of sequences are:  16443
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 50)             150000    
                                                                 
 lstm (LSTM)                 (None, 256)               314368    
                                                                 
 dense (Dense)               (None, 3000)              771000    
                                                                 
Total params: 1235368 (4.71 MB)
Trainable params: 1235368 (4.71 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
# Define callbacks
checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='loss', patience=5)

# Use generator to load data in batches (you can modify batch_size as needed)
def data_generator(X, y, batch_size):
    while True:
        indices = np.random.choice(len(X), batch_size, replace=False)
        yield X[indices], y[indices]

# Train the model using the data generator
batch_size = 64
epochs = 50

train_generator = data_generator(X, y, batch_size)
steps_per_epoch = len(X) // batch_size

model.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint, early_stopping])

Epoch 1/50
256/256 [==============================] - ETA: 0s - loss: 6.6394
Epoch 1: loss improved from inf to 6.63939, saving model to next_words.h5
256/256 [==============================] - 13s 41ms/step - loss: 6.6394
Epoch 2/50
  3/256 [..............................] - ETA: 8s - loss: 6.2312

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


256/256 [==============================] - ETA: 0s - loss: 6.1621
Epoch 2: loss improved from 6.63939 to 6.16214, saving model to next_words.h5
256/256 [==============================] - 9s 36ms/step - loss: 6.1621
Epoch 3/50
255/256 [============================>.] - ETA: 0s - loss: 6.0047
Epoch 3: loss improved from 6.16214 to 6.00361, saving model to next_words.h5
256/256 [==============================] - 10s 41ms/step - loss: 6.0036
Epoch 4/50
255/256 [============================>.] - ETA: 0s - loss: 5.7497
Epoch 4: loss improved from 6.00361 to 5.74950, saving model to next_words.h5
256/256 [==============================] - 10s 40ms/step - loss: 5.7495
Epoch 5/50
255/256 [============================>.] - ETA: 0s - loss: 5.5927
Epoch 5: loss improved from 5.74950 to 5.59167, saving model to next_words.h5
256/256 [==============================] - 11s 41ms/step - loss: 5.5917
Epoch 6/50
256/256 [==============================] - ETA: 0s - loss: 5.3811
Epoch 6: loss improved from

In [7]:
# Rest of the code remains the same
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break

  print(predicted_word)
  return predicted_word
while(True):
  text = input("Enter your line: ")

  if text == "0":
      print("Execution completed.....")
      break

  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)

          Predict_Next_Words(model, tokenizer, text)

      except Exception as e:
        print("Error occurred: ",e)
        continue

Enter your line: clearing up those
['clearing', 'up', 'those']
1/1 [==============================] - 0s 407ms/step
mysteries
Enter your line: twice in a dark
['in', 'a', 'dark']
1/1 [==============================] - 0s 21ms/step
silhouette
Enter your line: 0
Execution completed.....
